In [1]:
!pip install -q transformers datasets accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 2.3 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [3]:
# Configuration
model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"
dataset_name = "nvidia/OpenMathReasoning"
max_length = 512  # Reduced sequence length
use_4bit = True  # Enable 4-bit quantization

In [4]:
# Quantization setup
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [7]:
# Load model with quantization
# Check if CUDA is available before attempting 4-bit quantization
if use_4bit and not torch.cuda.is_available():
    print("CUDA is not available. Cannot use 4-bit quantization.")
    # Fallback to loading the model without quantization, or handle as needed
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto" # Keep device_map if it helps with multi-CPU threading or future GPU use
    )
elif use_4bit:
    # Load model with quantization if CUDA is available and 4-bit is requested
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )
else:
    # Load model without quantization if 4-bit is not requested
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto"
    )

# Only prepare for kbit training if quantization was applied
if use_4bit and torch.cuda.is_available():
    model = prepare_model_for_kbit_training(model)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


CUDA is not available. Cannot use 4-bit quantization.


model.safetensors.index.json:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.77G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Reduced target modules
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [9]:
# Load small dataset subset
try:
    dataset = load_dataset(dataset_name, split='train[:1000]')  # Small subset
except Exception as e:
    print(f"Dataset error: {e}")
    # Fallback to dummy data
    from datasets import Dataset
    dataset = Dataset.from_dict({
        "question": ["Solve: 2+2="],
        "answer": ["4"]
    })

README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/144 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/144 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

...

Generating cot split:   0%|          | 0/3201061 [00:00<?, ? examples/s]

Generating tir split:   0%|          | 0/1718466 [00:00<?, ? examples/s]

Generating genselect split:   0%|          | 0/565620 [00:00<?, ? examples/s]

Generating additional_problems split:   0%|          | 0/193170 [00:00<?, ? examples/s]

Dataset error: Unknown split "train". Should be one of ['cot', 'tir', 'genselect', 'additional_problems'].


In [16]:
!df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G  187G   39G  83% /


In [10]:
# Preprocessing function
def preprocess_function(examples):
    texts = [f"Question: {q}\nAnswer: {a}" for q, a in zip(examples["question"], examples["answer"])]
    tokenized = tokenizer(
        texts,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

In [19]:
# Process dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names,
    batch_size=4  # Small batch size
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Minimal batch size
    gradient_accumulation_steps=4,  # Compensate for small batch size
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=1,
    logging_steps=10,
    report_to="none"  # Disable external logging
)

In [13]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


ValueError: fp16 mixed precision requires a GPU (not 'xla').

In [ ]:
# Start training
trainer.train()